In [1]:
import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler

In [2]:


# Load dataset
data = pd.read_csv("combined_data.csv")


In [3]:

# Pre-processing
scaler = StandardScaler()
scaled_data = scaler.fit_transform(data)


In [4]:

# Feature engineering
features = pd.DataFrame(scaled_data, columns=["current_1", "current_2", "current_3"])
features["mean"] = features.mean(axis=1)
features["std_dev"] = features.std(axis=1)
features["range"] = features.max(axis=1) - features.min(axis=1)


In [5]:
features

,current_1,current_2,current_3,mean,std_dev,range
0,0.313208,1.068750,-1.452652,-0.023565,1.056544,2.521402
1,0.263676,1.090930,-1.401367,-0.015587,1.036461,2.492297
2,0.263676,1.118655,-1.397704,-0.005124,1.044735,2.516359
3,0.230655,1.129745,-1.394040,-0.011213,1.044429,2.523786
4,0.219648,1.168561,-1.394040,-0.001944,1.057846,2.562601
...,...,...,...,...,...,...
3169995,1.270832,-1.055003,-0.243787,-0.009319,0.963884,2.325834
3169996,1.287342,-1.016187,-0.280419,-0.003088,0.960641,2.303530
3169997,1.292846,-0.994007,-0.324378,-0.008513,0.959949,2.286853
3169998,1.303853,-0.955192,-0.357347,-0.002895,0.955702,2.259045


In [6]:

# Model selection and training
model = IsolationForest(n_estimators=100, contamination=0.01, random_state=42)
model.fit(features)

c:\Users\deepdesk\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(


IsolationForest(contamination=0.01, random_state=42)

In [12]:
import copy
import numpy as np

In [13]:
def add_anomalies(dataframe, number_of_anomalies=0, data_length=0):

        # Determine the upper bound of the data to which anomalies should be added.
        upper_bound = number_of_anomalies if number_of_anomalies else len(dataframe)
        
        # Determine the length of the data to which anomalies should be added.
        length = data_length if data_length else len(dataframe)

        # Make a deep copy of the data up to the upper bound and length.
        df = copy.deepcopy(dataframe[0:length])
        noise_df = df.iloc[:upper_bound,:]

        # Calculate the noise amplitude based on the signal amplitude.
        noise_amplitude = 0.1 * np.max(np.abs(df))

        # Add noise to the signal using amplitude scaling.
        noise = np.random.normal(0, noise_amplitude, noise_df.shape)
        
        df.iloc[:upper_bound,:] += noise

        return df


In [14]:
anomalous_data = add_anomalies(data,5000,10000)

c:\Users\deepdesk\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:84: FutureWarning: In a future version, DataFrame.max(axis=None) will return a scalar max over the entire DataFrame. To retain the old behavior, use 'frame.max(axis=0)' or just 'frame.max()'
  return reduction(axis=axis, out=out, **passkwargs)


In [15]:


# Anomaly detection
# new_data = pd.read_csv("real_time_data.csv")
scaled_new_data = scaler.transform(anomalous_data)
new_features = pd.DataFrame(scaled_new_data, columns=["current_1", "current_2", "current_3"])
new_features["mean"] = new_features.mean(axis=1)
new_features["std_dev"] = new_features.std(axis=1)
new_features["range"] = new_features.max(axis=1) - new_features.min(axis=1)

anomalies = model.predict(new_features)



In [18]:
sum(anomalies == -1)

3555

In [20]:
anomalies = model.predict(features[:10000])

In [21]:
sum(anomalies == -1)

78

In [ ]:

# Alert generation
if -1 in anomalies:
    print("Anomaly detected! Alert the operator.")
else:
    print("No anomalies detected.")

# Model evaluation
# Check the detected anomalies and their validity based on the domain knowledge and expertise.